# Part 2: The Data Engineer's Toolkit


In [ ]:
# Setup: Import required libraries
from pyspark.sql.functions import *

# Load TPC-H datasets (built into Databricks)
# Keep it simple: just two tables for joins
orders_df=spark.table("samples.tpch.orders") 
customers_df = spark.table("samples.tpch.customer")


## Module 2.1: Aggregating

**Goal**: Group and aggregate order data by status.

In [ ]:
# Simple aggregation: group by order status
order_stats = orders_df.groupBy("o_orderstatus").agg(
    count("o_orderkey").alias("order_count"),
    avg("o_totalprice").alias("avg_price"),
    sum("o_totalprice").alias("total_value")
)


order_stats.show()

## Module 2.2: Joining (15 mins)

**Goal**: Combine datasets to build a richer view.

### Join Types Explained
- **Inner Join**: Only matching rows from both tables
- **Left Join**: All rows from left table + matching rows from right (nulls for non-matches)
- **Right Join**: All rows from right table + matching rows from left
- **Full Outer Join**: All rows from both tables


In [ ]:
# Simple join: customers with their orders
joined_df = customers_df.join(
    orders_df,
    customers_df.c_custkey == orders_df.o_custkey,
    how="inner"
)

joined_df.select(
    "c_name",
    "c_mktsegment",
    "o_orderkey",
    "o_totalprice",
    "o_orderstatus"
).show(5)


### Left vs Inner Join

**Inner Join**: Only customers that have matching orders
```python
customers_df.join(orders_df, customers_df.c_custkey == orders_df.o_custkey, how="inner")
```

**Left Join**: All customers, even if they have no orders (nulls for customers without orders)
```python
customers_df.join(orders_df, customers_df.c_custkey == orders_df.o_custkey, how="left")
```

**When to use what?**
- Use **inner** when you only want complete records (customers with orders)
- Use **left** when you want to keep all records from the main table (all customers)


In [ ]:
# Compare inner vs left join
print("=== Inner Join (only customers with orders) ===")
inner_joined = customers_df.join(
    orders_df,
    customers_df.c_custkey == orders_df.o_custkey,
    how="inner"
)
print(f"Rows: {inner_joined.count():,}")

print("\n=== Left Join (all customers) ===")
left_joined = customers_df.join(
    orders_df,
    customers_df.c_custkey == orders_df.o_custkey,
    how="left"
)
print(f"Rows: {left_joined.count():,}")


## Module 2.3: Cleaning & Saving

**Goal**: Handle nulls and save our work to a reliable format (Delta Lake).


In [ ]:
# Handle nulls - drop rows with missing prices
cleaned_df = joined_df.dropna(subset=["o_totalprice"])

print(f"Original rows: {joined_df.count():,}")
print(f"Cleaned rows: {cleaned_df.count():,}")


### Delta Lake: A Better Parquet

**Delta Lake** provides:
- ✅ ACID transactions (atomicity, consistency, isolation, durability)
- ✅ Time travel (access previous versions)
- ✅ Schema enforcement
- ✅ Better performance optimizations

Think of it as "Parquet with superpowers"!


In [ ]:
# Save to Delta Lake format
# Note: Delta Lake requires delta-spark package. In Databricks it's pre-installed.
# For local use: pip install delta-spark

username = spark.sql("SELECT current_user()").collect()[0][0]
clean_username = username.split('@')[0].replace('.', '_').replace('-', '_')
table_name = f"{clean_username}_cleaned_orders"

# Save our cleaned DataFrame to a managed Delta Table.
# This is the preferred, modern Databricks method.
# It automatically manages the file path and avoids all DBFS access errors.
cleaned_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"✓ Saved to Delta Lake table: {table_name}")

In [ ]:
# Read it back to verify
read_back = spark.read.table(table_name)
read_back.display(5)

### Key Takeaways

1. **Aggregations**: Use `groupBy().agg()` for summary statistics
2. **Joins**: Choose the right join type for your use case
3. **Cleaning**: Handle nulls early in your pipeline
4. **Delta Lake**: Use for production pipelines (reliability + performance)
